In [9]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import osmnx as ox
import matplotlib.pyplot as plt
import numpy as np

In [2]:
tmc = pd.read_excel('../raw_data/Thailand_T19_v3.2_flat.xlsx')
tmc_gdf = gpd.GeoDataFrame(
    tmc,
    geometry=gpd.points_from_xy(tmc.LONGITUDE, tmc.LATITUDE),
    crs="EPSG:4326"
)

bangkok = ox.geocode_to_gdf("R92277", by_osmid=True)

tmc_utm = tmc_gdf.to_crs(32647)
bangkok_utm = bangkok.to_crs(32647)
poly_utm = bangkok_utm.geometry.iloc[0]

In [3]:
inside = tmc_utm[tmc_utm.within(poly_utm)]

In [4]:
node_df = inside[["LOCATION CODE", "ROAD NAME", "FIRST NAME", "SECOND NAME", "LATITUDE", "LONGITUDE"]]
node_df.rename({
    "LOCATION CODE": "loc_code", 
    "ROAD NAME": "road_name", 
    "FIRST NAME": "first_name", 
    "SECOND NAME": "second_name", 
    "LATITUDE": "lat", 
    "LONGITUDE": "lon"
}, inplace=True, axis="columns")

C:\Users\kongw\AppData\Local\Temp\ipykernel_15288\2457026138.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  node_df.rename({


In [7]:
node_df.to_csv("../processed_data/bkk_loc_table.csv", index=False)

In [62]:
connection_df = inside[["LOCATION CODE", "NEGATIVE OFFSET", "POSITIVE OFFSET", "LATITUDE", "LONGITUDE"]]

In [63]:
idx_left = np.isin(connection_df["NEGATIVE OFFSET"], connection_df["LOCATION CODE"])
idx_right = np.isin(connection_df["POSITIVE OFFSET"], connection_df["LOCATION CODE"])

In [64]:
edge_df = pd.DataFrame(columns=['start_loc', 'end_loc'])

In [65]:
temp_df = connection_df.loc[idx_left, ["NEGATIVE OFFSET", "LOCATION CODE"]]
temp_df = temp_df.rename({"NEGATIVE OFFSET":"start_loc", "LOCATION CODE":"end_loc"}, axis="columns").astype(np.int64)
temp_df = temp_df.astype(np.int64)
edge_df = pd.concat([edge_df, temp_df], ignore_index=True)

In [66]:
temp_df = connection_df.loc[idx_right, ["LOCATION CODE", "POSITIVE OFFSET"]]
temp_df = temp_df.rename({"POSITIVE OFFSET":"end_loc", "LOCATION CODE":"start_loc"}, axis="columns").astype(np.int64)
temp_df = temp_df.astype(np.int64)
edge_df = pd.concat([edge_df, temp_df], ignore_index=True)

In [68]:
edge_df = edge_df.drop_duplicates()

In [69]:
unique_point = connection_df[["LATITUDE", "LONGITUDE"]].drop_duplicates()

In [70]:
for idx, (lat, long) in unique_point.iterrows():
    temp_df = connection_df[["LOCATION CODE", "LATITUDE", "LONGITUDE"]]
    temp_df = temp_df[(temp_df["LATITUDE"] == lat) & (temp_df["LONGITUDE"] == long)]
    if temp_df.shape[0] > 1:
        new_df = pd.DataFrame({"start_loc":temp_df.iloc[:-1, 0].to_numpy(), "end_loc":temp_df.iloc[1:, 0].to_numpy()})
        edge_df = pd.concat([edge_df, new_df])

In [71]:
edge_df

,start_loc,end_loc
0,46141,46142
1,46142,46143
2,46143,46144
3,46144,46145
4,46145,46146
...,...,...
0,53417,53419
0,53430,53432
0,53436,53438
0,53667,53669


In [73]:
edge_df.to_csv("../processed_data/bkk_edge_table.csv", index=False)